# 🍳 PPO-LLM Strategy Shaping: Setup & Configuration

**Run this notebook first** to set up the environment before running any other notebooks.

## What this does:
- Installs required dependencies
- Sets up Overcooked-AI environment
- Mounts Google Drive for saving results
- Defines shared configuration

## Step 1: Clone Overcooked-AI Repository

In [ ]:
%cd /content

# Get a clean fresh copy of the HumanCompatibleAI overcooked repo
!rm -rf overcooked_ai
!git clone --depth 1 https://github.com/HumanCompatibleAI/overcooked_ai.git /content/overcooked_ai > /dev/null

import sys, os
sys.path.append("/content/overcooked_ai/src")

print("CWD:", os.getcwd())
print("overcooked_ai in path:", "/content/overcooked_ai/src" in sys.path)

## Step 2: Install Dependencies

In [ ]:
!pip install -q gym "stable-baselines3[extra]" overcooked-ai transformers

In [ ]:
# Fix NumPy compatibility (run this cell, then restart runtime)
!pip uninstall -y numpy
!pip install numpy==1.26.4

import os
os._exit(0)  # Forces runtime restart - RE-RUN CELLS ABOVE after this

In [ ]:
# Add shimmy for stable-baselines3 + gym compatibility
!pip install -q "shimmy>=2.0"

## Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import os, zipfile

DRIVE_ROOT = "/content/drive/MyDrive"
print("Contents of MyDrive:", os.listdir(DRIVE_ROOT))

# Prefer the already-unzipped runs folder if it exists
RUNS_ROOT = os.path.join(DRIVE_ROOT, "runs")

# If runs folder is not there but runs.zip is, unzip once
if (not os.path.exists(RUNS_ROOT)) and os.path.exists(os.path.join(DRIVE_ROOT, "runs.zip")):
    print("Unzipping runs.zip into MyDrive...")
    with zipfile.ZipFile(os.path.join(DRIVE_ROOT, "runs.zip"), "r") as zf:
        zf.extractall(DRIVE_ROOT)

print("Using RUNS_ROOT:", RUNS_ROOT, "exists:", os.path.exists(RUNS_ROOT))

## Step 4: Shared Configuration

These values are used across all notebooks. Modify here to change experiment settings.

In [ ]:
import torch
from overcooked_ai_py.mdp.actions import Action

# ==========================================
# SHARED CONFIGURATION - EDIT HERE
# ==========================================

# Environment settings
LAYOUT = "asymmetric_advantages"  # Options: "cramped_room", "asymmetric_advantages", "coordination_ring"
HORIZON = 400  # Episode length

# Hardware
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Experiment seeds for reproducibility
SEEDS = [1001, 2002, 3003, 4004, 5005]

# Training settings
CHECKPOINT_EVERY_STEPS = 50_000
LOG_EVERY_STEPS = 2_048
EVAL_EPISODES = 10

# File paths
RESULTS_CSV = "/content/drive/MyDrive/results_combined_new.csv"
RUNS_DIR = "/content/drive/MyDrive/runs"

# LLM reward shaping
LLM_BONUS = 0.2
LLM_MODEL_NAME = "EleutherAI/gpt-neo-1.3B"  # or "EleutherAI/gpt-neo-125M" for faster

# Training steps per baseline
BASELINE_STEPS = {
    "Baseline": 1_000_000,
    "CC_PPO":   1_000_000,
    "SP_PPO":   1_000_000,
    "HARL":     1_000_000,
    "PPO+LLM":  600_000,   # LLM shaping requires fewer steps!
    "PBT_PPO":  1_000_000,
}

# Baselines and environments to test
BASELINES = ["Baseline", "PPO+LLM", "CC_PPO", "SP_PPO", "HARL", "PBT_PPO"]
ENV_NAMES = ["No Noise", "Noise", "Delay", "Combo"]

# Derived
NUM_ACTIONS = len(Action.ALL_ACTIONS)

print("\n✅ Configuration loaded!")
print(f"Layout: {LAYOUT}")
print(f"Baselines: {BASELINES}")
print(f"Environments: {ENV_NAMES}")
print(f"Seeds: {SEEDS}")

## ✅ Setup Complete!

You can now run other notebooks:
- **02_training.ipynb** - Train all baselines
- **03_nash_analysis.ipynb** - Nash gap analysis
- **04_latency_analysis.ipynb** - Measure inference latency
- **05_robustness_analysis.ipynb** - Test robustness to perturbations
- **06_task_completion.ipynb** - Evaluate task completion metrics